In [99]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re


In [100]:
url = 'http://kmitl.ac.th/school-engineering'
en_url = 'http://kmitl.ac.th/school-engineering'

In [101]:
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-41909381-25">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];function gtag(){dataLayer.push(arguments)};gtag("js", new Date());gtag("config", "UA-41909381-25", {"groups":"default","anonymize_ip":true});
  </script>
  <link href="http://kmitl.ac.th/school-engineering" rel="canonical"/>
  <meta content="หลักสูตรทั้งหมดของคณะ" name="description"/>
  <meta content="Drupal 9 (https://www.drupal.org)" name="Generator"/>
  <meta content="width" name="MobileOpti

In [102]:
def create_dataframe (site_url) :
    why_this = []
def create_dataframe (site_url) :
    why_this = []
    data  = requests.get(site_url).text
    soup = BeautifulSoup(data, 'html.parser')
    df = pd.DataFrame(columns=["Faculty", "Major", "Link","Campus","IsInter","WhyApply","Fee","Degree"])
    divs_rows = soup.find_all("div", {"class":"row three-col"})
    for  row in divs_rows : 
        elements_in_row = row.find_all("p",{"class": "border border-danger p-2"})
        for ele in elements_in_row:
        #print(temp.get_text())
            campus  = 'KMITL'
            inter  = 'False'
            temp = ele.find("a")
            link = temp.get('href')
            text = temp.get_text()
            splitted_text = text.split("\n")
            temp3 = re.search('เขต',text)
            temp4 = re.search('Inter',text)            
            if(temp3 == None):
                campus= 'KMITL'
            else : 
                campus = 'Chumporn'
                
            if(temp4 != None) : 
                inter = "True"
            print(text)
            faculty = re.search(r'\n(.*?)$',text).group(1)
            major = re.search(r'\n(.*?)\n',text).group(1)
            # Go to another page for scraping more information
            data2  = requests.get(link).text
            soup2= BeautifulSoup(data2, 'html.parser')
            temp_div = soup2.find("div", {"class":"clearfix text-formatted field field--name-field-description field--type-text-with-summary field--label-hidden field__item"})
            text_arr  = temp_div.get_text()
            degree = re.findall(r'\b[ก-ฮ.]+\.[บดม]\.\s',text_arr)
    
            whyapply = re.search(r'\?\n(.*?)\n',text_arr).group(1)
        
            fee_div = soup2.find("div",{"class":"clearfix text-formatted field field--name-field-price field--type-text field--label-hidden field__item"})
            fee = fee_div.get_text()
            if(len(degree) == 0 ) : 
                degree =  '-'
            else : 
                degree =  degree[0]
            if(len(whyapply ) == 0 ) : 
                whyapply  =  '-'
            print(degree)
            
            df =  df.append({"Faculty" : faculty, "Major":major, "Link":link , "Campus":campus,'IsInter' : inter,
                             'WhyApply':whyapply,"Fee":fee, "Degree":degree},  ignore_index=True)    
    return df 


In [111]:
def run_jsonifier(df):
    # convert index values to string (when they're something else)
    df.index = df.index.map(str)
    # convert column names to string (when they're something else)
    df.columns = df.columns.map(str)

    # convert DataFrame to dict and dict to string
    js = str(df.to_dict())
    #store indices of double quote marks in string for later update
    idx = [i for i, _ in enumerate(js) if _ == '"']
    # jsonify quotes from single to double quotes  
    js = js.replace("'", '"')
    # add \ to original double quotes to make it json-like escape sequence 
    for add, i in enumerate(idx):
        js = js[:i+add] + '\\' + js[i+add:] 
    return js



# run our function to convert dataframe to json
print(run_jsonifier(temp_df))


{"Faculty": {"0": "วิทยาเขตชุมพรเขตรอุดมศักดิ์ จังหวัดชุมพร ", "1": "วิทยาเขตชุมพรเขตรอุดมศักดิ์ จังหวัดชุมพร"}, "Major": {"0": "บริหารธุรกิจและการเป็นผู้ประกอบการ", "1": "นวัตกรรมอาหารและการจัดการ"}, "Link": {"0": "http://www-uat.kmitl.ac.th/node/556", "1": "http://www-uat.kmitl.ac.th/node/557"}, "Campus": {"0": "Chumporn", "1": "Chumporn"}, "IsInter": {"0": "False", "1": "False"}, "WhyApply": {"0": "ทักษะในการบริหารธุรกิจและบริการพร้อมด้วยคุณธรรมจริยธรรมและมีจิตอาสาเพื่อผลิตบัณฑิตที่มีความรู้คู่คุณธรรมและตอบสนองต่อการพัฒนาประเทศในด้านการสร้างผู้ประกอบการยุคใหม่และมีองค์ความรู้และสามารถ ขณะเดียวกันสามารถใช้เทคโนโลยีสารสนเทศในการบริหารจัดการให้เกิดประสิทธิภาพสูงสุด พร้อมทั้งยกระดับการพัฒนาของประเทศสู่ธุรกิจที่ยั่งยืนและมีศักยภาพทางการแข่งขันให้เกิดการกระจายรายได้ การสร้างอาชีพ และการสร้างคุณภาพชีวิตที่ดีให้แก่สังคม", "1": "นวัตกรรมและการบริหารจัดการธุรกิจทางด้านอาหาร โดยบูรณาการความรู้ทางด้านวิทยาศาสตร์ประยุกต์ เทคโนโลยี นวัตกรรมอาหาร และการจัดการ เข้าด้วยกัน เพื่อนำความรู้ไปประยุกต์ใช

In [122]:
json  =run_jsonifier(temp_df)

In [123]:
with open('df.json', 'w', encoding='utf-8') as file:
    temp_df.to_json(file, force_ascii=False,orient = 'records')

In [124]:
data3  = requests.get('http://www-uat.kmitl.ac.th/parents-and-prospective-students').text
soup3 = BeautifulSoup(data3, 'html.parser')
over_all = soup3.find_all("div", {"col-md-6 col-lg-6 col-second"})
over_all = over_all[0]

In [128]:

for site in over_all.find_all('a') : 
    link  =  'http://www-uat.kmitl.ac.th' + site.get('href')
    filename = site.get_text()
    temp_df = create_dataframe(link)
#     temp_df.dropna(axis=0, inplace=True)
    
    with open(filename +'.json', 'w', encoding='utf-8') as file:
        temp_df.to_json(file, force_ascii=False,orient = 'records')
    

วศ.บ.
วิศวกรรมโทรคมนาคมและโครงข่าย
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมไฟฟ้า
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมอิเล็กทรอนิกส์
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมคอมพิวเตอร์
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมคอมพิวเตอร์ (ต่อเนื่อง)
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมเครื่องกล
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมขนส่งทางราง
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมการวัดคุม (ต่อเนื่อง)
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมเมคคาทรอนิกส์และออโตเมชัน
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมโยธา
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมโยธา (ต่อเนื่อง)
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมระบบอุตสาหกรรมการเกษตร (ต่อเนื่อง)
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมเกษตรอัจฉริยะ
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมเคมี
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
วิศวกรรมอุตสาหการ
คณะวิศวกรรมศาสตร์w
วศ.บ. 
วศ.บ.
วิศวกรรมอาหาร
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
B.Eng. Civil Engineering (International Program)
คณะวิศวกรรมศาสตร์
วศ.บ. 
วศ.บ.
B.Eng. Biomedical Engineering (Internation Progra

In [127]:
 temp_df

,Faculty,Major,Link,Campus,IsInter,WhyApply,Fee,Degree
0,วิทยาเขตชุมพรเขตรอุดมศักดิ์ จังหวัดชุมพร,บริหารธุรกิจและการเป็นผู้ประกอบการ,http://www-uat.kmitl.ac.th/node/556,Chumporn,False,ทักษะในการบริหารธุรกิจและบริการพร้อมด้วยคุณธรร...,"16,000",บธ.บ.
1,วิทยาเขตชุมพรเขตรอุดมศักดิ์ จังหวัดชุมพร,นวัตกรรมอาหารและการจัดการ,http://www-uat.kmitl.ac.th/node/557,Chumporn,False,นวัตกรรมและการบริหารจัดการธุรกิจทางด้านอาหาร โ...,"19,000",วท.บ.
